In [14]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import f1_score

### https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia

In [2]:
train_data = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255.0
    ,brightness_range=(1.5,2)
    ,horizontal_flip=True
)

val_data  = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255.0
    ,brightness_range=(1.5,2)
    ,horizontal_flip=True
)

test_data  = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255.0
    ,brightness_range=(1.5,2)
    ,horizontal_flip=True
)

In [3]:
train = train_data.flow_from_directory(
    '/kaggle/input/chest-xray-pneumonia/chest_xray/train',
    target_size=(224,224)
    ,class_mode='binary'
)
val = val_data.flow_from_directory(
    '/kaggle/input/chest-xray-pneumonia/chest_xray/val',
    target_size=(224,224)
    ,class_mode='binary'
)
test = test_data.flow_from_directory(
    '/kaggle/input/chest-xray-pneumonia/chest_xray/test',
    target_size=(224,224)
    ,class_mode='binary'
)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [16]:
mobilenet = tf.keras.applications.MobileNet(
    input_shape=(224,224,3)
    ,include_top=False
    ,pooling='max'
)
mobilenet.trainable = False

In [20]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(64,kernel_size = (3,3),activation='relu',input_shape=(224,224,3)))
model.add(tf.keras.layers.Conv2D(64,kernel_size = (3,3),activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(padding='same',strides=(2,2)))

model.add(tf.keras.layers.Conv2D(128,kernel_size = (3,3),activation='relu'))
model.add(tf.keras.layers.Conv2D(128,kernel_size = (3,3),activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(padding='same',strides=(2,2)))

model.add(tf.keras.layers.Conv2D(256,kernel_size = (3,3),activation='relu'))
model.add(tf.keras.layers.Conv2D(256,kernel_size = (3,3),activation='relu'))
model.add(tf.keras.layers.Conv2D(256,kernel_size = (3,3),activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(padding='same',strides=(2,2)))

model.add(tf.keras.layers.Conv2D(512,kernel_size = (3,3),activation='relu'))
model.add(tf.keras.layers.Conv2D(512,kernel_size = (3,3),activation='relu'))
model.add(tf.keras.layers.Conv2D(512,kernel_size = (3,3),activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(padding='same',strides=(2,2)))

model.add(tf.keras.layers.Conv2D(512,kernel_size = (3,3),activation='relu'))
model.add(tf.keras.layers.Conv2D(512,kernel_size = (3,3),activation='relu'))
model.add(tf.keras.layers.Conv2D(512,kernel_size = (3,3),activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(padding='same',strides=(2,2)))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(4096,activation='relu'))
model.add(tf.keras.layers.Dense(1000,activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [21]:

with tf.device('/GPU:0'):
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.AUC(name='auc'),'accuracy']
    )

    hist = model.fit(
        train,
        validation_split = 0.3,
        validation_data=test,
        batch_size = 30,
        epochs=100,
        callbacks=tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            restore_best_weights=True,
            patience=5
        )
    )

Epoch 1/100
163/163 [==============================] - 78s 404ms/step - loss: 0.6873 - auc: 0.5005 - accuracy: 0.7375 - val_loss: 0.7311 - val_auc: 0.5000 - val_accuracy: 0.6250
Epoch 2/100
163/163 [==============================] - 63s 387ms/step - loss: 0.5723 - auc: 0.4960 - accuracy: 0.7429 - val_loss: 0.6902 - val_auc: 0.5000 - val_accuracy: 0.6250
Epoch 3/100
163/163 [==============================] - 63s 387ms/step - loss: 2.0727 - auc: 0.4815 - accuracy: 0.7295 - val_loss: 0.7891 - val_auc: 0.6487 - val_accuracy: 0.6250
Epoch 4/100
163/163 [==============================] - 63s 385ms/step - loss: 0.5742 - auc: 0.4973 - accuracy: 0.7429 - val_loss: 0.6899 - val_auc: 0.5000 - val_accuracy: 0.6250
Epoch 5/100
163/163 [==============================] - 62s 379ms/step - loss: 0.5718 - auc: 0.4863 - accuracy: 0.7429 - val_loss: 0.6862 - val_auc: 0.5000 - val_accuracy: 0.6250
Epoch 7/100
163/163 [==============================] - 62s 377ms/step - loss: 0.5706 - auc: 0.5046 - accuracy:

In [22]:
labels = test.labels
predict = np.squeeze(model.predict(test) >= 0.5).astype(np.int)

20/20 [==============================] - 6s 305ms/step


/tmp/ipykernel_32/1316839609.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  predict = np.squeeze(model.predict(test) >= 0.5).astype(np.int)


In [23]:
f1_score(labels,predict)

0.7692307692307693

In [25]:
inputs = tf.keras.Input(shape=(224,224,3))
mobilenet_m = mobilenet(inputs,training=False)

dense_1 = tf.keras.layers.Dense(1024,activation='relu')(mobilenet_m)
outputs = tf.keras.layers.Dense(1,activation='sigmoid')(dense_1)

model = tf.keras.Model(inputs=inputs,outputs=outputs)


with tf.device('/GPU:0'):
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.AUC(name='auc'),'accuracy']
    )

    hist = model.fit(
        train,
        validation_split = 0.3,
        validation_data=test,
        batch_size = 30,
        epochs=100,
        callbacks=tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            restore_best_weights=True,
            patience=5
        )
    )
    
labels = test.labels
predict = np.squeeze(model.predict(test) >= 0.5).astype(np.int)
    
f1_score(labels,predict)

Epoch 1/100
163/163 [==============================] - 63s 374ms/step - loss: 0.3917 - auc: 0.9510 - accuracy: 0.9153 - val_loss: 0.6083 - val_auc: 0.9230 - val_accuracy: 0.8285
Epoch 2/100
163/163 [==============================] - 61s 373ms/step - loss: 0.1205 - auc: 0.9872 - accuracy: 0.9559 - val_loss: 0.4862 - val_auc: 0.9343 - val_accuracy: 0.8654
Epoch 5/100
163/163 [==============================] - 60s 367ms/step - loss: 0.0859 - auc: 0.9935 - accuracy: 0.9655 - val_loss: 0.3810 - val_auc: 0.9390 - val_accuracy: 0.8878
Epoch 6/100
163/163 [==============================] - 60s 367ms/step - loss: 0.0853 - auc: 0.9933 - accuracy: 0.9680 - val_loss: 0.4864 - val_auc: 0.9523 - val_accuracy: 0.8462
Epoch 7/100
163/163 [==============================] - 60s 367ms/step - loss: 0.0846 - auc: 0.9936 - accuracy: 0.9666 - val_loss: 1.0911 - val_auc: 0.8946 - val_accuracy: 0.7372
Epoch 8/100
163/163 [==============================] - 59s 364ms/step - loss: 0.0866 - auc: 0.9932 - accuracy:

/tmp/ipykernel_32/1240158828.py:31: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  predict = np.squeeze(model.predict(test) >= 0.5).astype(np.int)


0.6415552855407046